In [11]:
import utils
import dataset

import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tflearn as tfl
import skimage as ski

In [33]:
importlib.reload(utils)
importlib.reload(dataset)
print()

In [13]:
%matplotlib inline

SIZE = 128
CATS = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
NUM_LABELS = len(CATS)

# Data Preprocesing

* remove duplicated files

# Feature extraction

In [14]:
X, Y = dataset.load_train_data(categories=CATS, size=SIZE)

Status: Processing... ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'] 
ALB: 961 files 
BET: 200 files 
DOL: 117 files 
LAG: 67 files 
NoF: 465 files 
OTHER: 299 files 
SHARK: 176 files 
YFT: 734 files 
Status: DONE 
X shape: (3019, 128, 128, 3) 
X size: 579648.0 
Y shape: (3019,) 
Y size: 23.5859375 


# Feature selection

In [25]:
log('Features', X.shape)
log('Labels', Y[0])

Features: (3019, 128, 128, 3) 
Labels: 0 


# Models

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras import utils

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(256, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(len(TYPES), activation='softmax')(x)

# this is the model we will train
inceptionv3 = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
inceptionv3.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
Y = utils.to_categorical(y)
inceptionv3.fit(X, Y, validation_split=0.1)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
# for i, layer in enumerate(base_model.layers):
#   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in inceptionv3.layers[:172]:
   layer.trainable = False
for layer in inceptionv3.layers[172:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
inceptionv3.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
inceptionv3.fit(X, Y, validation_split=0.2)

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

img_path = 'output/train/ALB/img_00003.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
log('X', x.shape)


resnet50 = ResNet50(weights='imagenet')
preds = resnet50.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])
# Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]

In [ ]:
from keras.applications.inception_v3 import preprocess_input, decode_predictions

img_path = 'output/train/SHARK/img_00365.jpg'
img = image.load_img(img_path, target_size=(SIZE, SIZE))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)
log('X', x.shape)

preds = inceptionv3.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
log('Predicted:', preds)
log('Predicted:', TYPES[preds.argmax()])

In [22]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

model = Sequential()

model.add(Conv2D(10, (5, 5), input_shape=X[0].shape))
model.add(Activation('relu'))
model.add(Conv2D(10, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_LABELS, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

YH = keras.utils.to_categorical(Y, num_classes=NUM_LABELS)
history = model.fit(X, YH, validation_split=0.1, epochs=6)
model.save('fisheries.h5')

Train on 2717 samples, validate on 302 samples
Epoch 1/6
2717/2717 [==============================] - 95s - loss: 1.9936 - acc: 0.5182 - val_loss: 0.7630 - val_acc: 0.7980

ImportError: `save_model` requires h5py.

In [118]:
import vis
from vis.visualization import visualize_cam
from keras.utils import plot_model

# heatmap = visualize_cam(model, 12, [pred_class], img)
# plt.imshow(heatmap)
# plot_model(model, to_file='fisheries.png')


# Test dataset

In [ ]:
T1, filenames1 = dataset.load_test1_data()
T2, filenames2 = dataset.load_test2_data()

Status: Processing... 12153 files 
Status: DONE 


In [148]:
len(filenames)

1000

# Prediction

In [28]:
# P1 = model.predict(T1)
# log('Predictions 1', P1.shape)

P2 = model.predict(T2)
log('Predictions 2', P2.shape)

Predictions: (1000, 8) 


In [31]:
P[-1]

array([  9.07867432e-01,   2.64520233e-04,   8.93137767e-05,
         6.61501195e-03,   4.20914683e-03,   7.35373497e-02,
         1.59240808e-05,   7.40129454e-03], dtype=float32)

# Submission

In [32]:
# sample = pd.read_csv('input/sample_submission_stg1.csv')
# print(sample.shape)

P = P2
filenames = filenames

submission = pd.DataFrame(columns=['image','ALB','BET','DOL','LAG','NoF','OTHER','SHARK','YFT'])
submission.image = filenames
submission.ALB = P[:, 0]
submission.BET = P[:, 1]
submission.DOL = P[:, 2]
submission.LAG = P[:, 3]
submission.NoF = P[: ,4]
submission.OTHER = P[:, 5]
submission.SHARK = P[:, 6]
submission.YFT = P[:, 7]
submission.head()

submission.to_csv('submission2.csv', index=False)

# References

* https://deepsense.io/deep-learning-right-whale-recognition-kaggle/
* http://matthewearl.github.io/2016/05/06/cnn-anpr/
* http://cs231n.stanford.edu/reports/2016/pdfs/285_Report.pdf
* https://www.kaggle.com/narae78/the-nature-conservancy-fisheries-monitoring/fish-detection/run/461339
